In [ ]:
import datetime
import numpy as np
import math
import os
import re
import sys
import matplotlib.pyplot as plt
import pandas as pd
from collections import namedtuple
from scipy import signal
from scipy.spatial.transform import Rotation as R
from typing import List, Tuple, cast

%reload_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")

# flags to change some script behaviour
use_lse = True      # To switch (off->False, ON->True) LSE plots
use_gnss = True     # To switch (off->False, ON->True) LSE plots
use_se = True       # To switch (off->False, ON->True) LSE plots

cov_scaling = 3     # 3-sigma values or x-sigma values
rcov_scaling = 3    # 3-sigma values or x-sigma values



In [ ]:
# Path to configure based on the data location
# Provide the URL to the single database only, if multiple given, it will take first value as input
workspace_path = "/path/to/workspace"
gnss_csv_path = os.path.join(workspace_path, "gnss.csv")
lse_csv_path = os.path.join(workspace_path, "lse.csv")
se_csv_path = os.path.join(workspace_path, "se.csv")

In [ ]:
gnss_df = None
lse_df = None
se_df = None

if use_gnss and os.path.isfile(gnss_csv_path):
  gnss_df = pd.read_csv(gnss_csv_path)
  print("Read Success:", gnss_csv_path)
else:
  if use_gnss:
    print("Read Failed:", gnss_csv_path)

if use_lse and os.path.isfile(lse_csv_path):
  lse_df = pd.read_csv(lse_csv_path)
  print("Read Success:", lse_csv_path)
else:
  if use_lse:
    print("Read Failed:", lse_csv_path)

if use_se and os.path.isfile(se_csv_path):
  se_df = pd.read_csv(se_csv_path)
  print("Read Success:", se_csv_path)
else:
  if use_se:
    print("Read Failed:", se_csv_path)

In [ ]:
def calc_euler(row):
  r = R.from_quat([row['qx'], row['qy'], row['qz'], row['qw']])
  r_eul = r.as_euler('xyz')
  row['rot_x_eu'] = np.rad2deg(r_eul[0])
  row['rot_y_eu'] = np.rad2deg(r_eul[1])
  row['rot_z_eu'] = np.rad2deg(r_eul[2])
  return row

if gnss_df is not None:
  gnss_df = gnss_df.apply(calc_euler, axis=1)

if lse_df is not None:
  lse_df = lse_df.apply(calc_euler, axis=1)

if se_df is not None:
  se_df = se_df.apply(calc_euler, axis=1)

In [ ]:
# Perform GNSS dataframe custom data manipulation if required
if gnss_df is not None:
  pass

In [ ]:
# Perform LSE dataframe custom data manipulation if required
if lse_df is not None:
  pass

In [ ]:
# Perform SE dataframe custom data manipulation if required
if se_df is not None:
  pass

In [ ]:
%matplotlib widget

fig, axs = plt.subplots(3, 1, sharex=True, figsize=(10, 10))
fig.subplots_adjust(hspace=0.2)
plt.xlabel("timestamp")
plt.suptitle(str(cov_scaling)+'-$\sigma$ Position Plots: ')

for idx,axis in enumerate(['x','y','z']):
  axs[idx].set_ylabel(axis.upper()+" (m)")
  if gnss_df is not None:
    axs[idx].plot(gnss_df['timestamp'], gnss_df[axis], color ="red", lw=2, label="GNSS")
    # axs[idx].plot(gnss_df['ts'], gnss_df[axis+'+'], color ="red",alpha=0.1)
    # axs[idx].plot(gnss_df['ts'], gnss_df[axis+'-'], color ="red", alpha=0.1)
    # axs[idx].fill_between(gnss_df['ts'], gnss_df['pos_'+axis], gnss_df[axis+'+'], alpha=0.1, color='red')
    # axs[idx].fill_between(gnss_df['ts'], gnss_df['pos_'+axis], gnss_df[axis+'-'], alpha=0.1, color='red')

  if lse_df is not None:
    axs[idx].plot(lse_df['timestamp'], lse_df[axis], color ="blue", lw=2, label="LSE")
    # axs[idx].plot(lse_df['ts'], lse_df[axis+'+'], color ="blue",alpha=0.1)
    # axs[idx].plot(lse_df['ts'], lse_df[axis+'-'], color ="blue",alpha=0.1)
    # axs[idx].fill_between(lse_df['ts'], lse_df['pos_'+axis], lse_df[axis+'+'], alpha=0.1, color='blue')
    # axs[idx].fill_between(lse_df['ts'], lse_df['pos_'+axis], lse_df[axis+'-'], alpha=0.1, color='blue')

  if se_df is not None:
    axs[idx].plot(se_df['timestamp'], se_df[axis], color ="green", lw=2, label="SE")
    # axs[idx].plot(se_df['ts'], se_df[axis+'+'], color ="green",alpha=0.1)
    # axs[idx].plot(se_df['ts'], se_df[axis+'-'], color ="green",alpha=0.1)
    # axs[idx].fill_between(se_df['ts'], se_df['pos_'+axis], se_df[axis+'+'], alpha=0.1, color='green')
    # axs[idx].fill_between(se_df['ts'], se_df['pos_'+axis], se_df[axis+'-'], alpha=0.1, color='green')

plt.legend(bbox_to_anchor=(1.07, 2), loc='upper center', ncol=1, fontsize='xx-small')
plt.show()

In [ ]:
%matplotlib widget

fig, qaxs = plt.subplots(3, 1, sharex=True, figsize=(10, 10))
fig.subplots_adjust(hspace=0.2)
plt.xlabel("timestamp")

plt.suptitle(str(rcov_scaling)+'-$\sigma$ Orientation Plots: ')

for idx,axis in enumerate(['rot_x_eu','rot_y_eu','rot_z_eu']):
  qaxs[idx].set_ylabel(rotmap[axis])
  if gnss_df is not None:
    qaxs[idx].plot(gnss_df['timestamp'], gnss_df[axis], color="red", lw=2, label="GNSS")
    qaxs[idx].plot(gnss_df['timestamp'], gnss_df[axis+'+'], color ="red",alpha=0.1)
    qaxs[idx].plot(gnss_df['timestamp'], gnss_df[axis+'-'], color ="red", alpha=0.1)
    qaxs[idx].fill_between(gnss_df['timestamp'], gnss_df['pos_'+axis], gnss_df[axis+'+'], alpha=0.1, color='red')
    qaxs[idx].fill_between(gnss_df['timestamp'], gnss_df['pos_'+axis], gnss_df[axis+'-'], alpha=0.1, color='red')

  if lse_df is not None:
    qaxs[idx].plot(lse_df['timestamp'], lse_df[axis], color="blue", lw=2, label="LSE")
    qaxs[idx].plot(lse_df['ts'], lse_df[axis+'+'], color ="blue",alpha=0.1)
    qaxs[idx].plot(lse_df['ts'], lse_df[axis+'-'], color ="blue",alpha=0.1)
    qaxs[idx].fill_between(lse_df['ts'], lse_df['pos_'+axis], lse_df[axis+'+'], alpha=0.1, color='blue')
    qaxs[idx].fill_between(lse_df['ts'], lse_df['pos_'+axis], lse_df[axis+'-'], alpha=0.1, color='blue')

  if se_df is not None:
    qaxs[idx].plot(se_df['timestamp'], se_df[axis], color="green", lw=2, label="SE")
    qaxs[idx].plot(se_df['ts'], se_df[axis+'+'], color ="green",alpha=0.1)
    qaxs[idx].plot(se_df['ts'], se_df[axis+'-'], color ="green",alpha=0.1)
    qaxs[idx].fill_between(se_df['ts'], se_df['pos_'+axis], se_df[axis+'+'], alpha=0.1, color='green')
    qaxs[idx].fill_between(se_df['ts'], se_df['pos_'+axis], se_df[axis+'-'], alpha=0.1, color='green')

plt.legend(bbox_to_anchor=(1.07, 2), loc='upper center', ncol=1, fontsize='xx-small')
plt.show()

In [ ]:
## ALL Pose plots in a single subplot ##
%matplotlib widget

fig, axs = plt.subplots(6, 1, sharex=True, figsize=(10, 10))
fig.subplots_adjust(hspace=0.2)
plt.xlabel("timestamp")
plt.suptitle(str(cov_scaling)+'-$\sigma$ Position Plots: ')

rotmap = {'rot_x_eu':'roll (deg)', 'rot_y_eu':'pitch (deg)', 'rot_z_eu':'yaw (deg)'}

for idx,axis in enumerate(['x','y','z']):
  orient_idx = 3 + idx
  axs[idx].set_ylabel(axis.upper()+" (m)")
  axs[orient_idx].set_ylabel(rotmap['rot_'+axis+'_eu'])

  if gnss_df is not None:
    axs[idx].plot(gnss_df['timestamp'], gnss_df[axis], color ="red", lw=2, label="GNSS")
    # axs[idx].plot(gnss_df['ts'], gnss_df[axis+'+'], color ="red",alpha=0.1)
    # axs[idx].plot(gnss_df['ts'], gnss_df[axis+'-'], color ="red", alpha=0.1)
    # axs[idx].fill_between(gnss_df['ts'], gnss_df['pos_'+axis], gnss_df[axis+'+'], alpha=0.1, color='red')
    # axs[idx].fill_between(gnss_df['ts'], gnss_df['pos_'+axis], gnss_df[axis+'-'], alpha=0.1, color='red')

    axs[orient_idx].plot(gnss_df['timestamp'], gnss_df['rot_'+axis+'_eu'], color="red", lw=2, label="GNSS")

  if lse_df is not None:
    axs[idx].plot(lse_df['timestamp'], lse_df[axis], color ="blue", lw=2, label="LSE")
    # axs[idx].plot(lse_df['ts'], lse_df[axis+'+'], color ="blue",alpha=0.1)
    # axs[idx].plot(lse_df['ts'], lse_df[axis+'-'], color ="blue",alpha=0.1)
    # axs[idx].fill_between(lse_df['ts'], lse_df['pos_'+axis], lse_df[axis+'+'], alpha=0.1, color='blue')
    # axs[idx].fill_between(lse_df['ts'], lse_df['pos_'+axis], lse_df[axis+'-'], alpha=0.1, color='blue')

    axs[orient_idx].plot(lse_df['timestamp'], lse_df['rot_'+axis+'_eu'], color="blue", lw=2, label="LSE")

  if se_df is not None:
    axs[idx].plot(se_df['timestamp'], se_df[axis], color ="green", lw=2, label="SE")
    # axs[idx].plot(se_df['ts'], se_df[axis+'+'], color ="green",alpha=0.1)
    # axs[idx].plot(se_df['ts'], se_df[axis+'-'], color ="green",alpha=0.1)
    # axs[idx].fill_between(se_df['ts'], se_df['pos_'+axis], se_df[axis+'+'], alpha=0.1, color='green')
    # axs[idx].fill_between(se_df['ts'], se_df['pos_'+axis], se_df[axis+'-'], alpha=0.1, color='green')

    axs[orient_idx].plot(se_df['timestamp'], se_df['rot_'+axis+'_eu'], color="green", lw=2, label="SE")

plt.legend(bbox_to_anchor=(1.07, 6), loc='upper center', ncol=1, fontsize='xx-small')
plt.show()

In [ ]:
covxy_fig = plt.figure(figsize=(10,15))

plt.title("X-Y plot")
plt.xlabel("X (m)")
plt.ylabel("Y (m)")

if gnss_df is not None:
  plt.plot(gnss_df['x'], gnss_df['y'], color ="red", label="GNSS")
  # plt.plot(poscov_df['x+'], poscov_df['y+'], color ="orange", label="pos+$\sigma$",alpha=0.3, linestyle='dashed')
  # plt.plot(poscov_df['x-'], poscov_df['y-'], color ="orange", label="pos-$\sigma$", alpha=0.3, linestyle='dashed')

if lse_df is not None:
  plt.plot(lse_df['x'], lse_df['y'], color ="blue", label="LSE")
  # plt.plot(lse_df['x+'], lse_df['y+'], color ="cyan", label="pos+$\sigma$",alpha=0.3, linestyle='dashed')
  # plt.plot(lse_df['x-'], lse_df['y-'], color ="cyan", label="pos-$\sigma$", alpha=0.3, linestyle='dashed')

if se_df is not None:
  plt.plot(se_df['x'], se_df['y'], color ="green", label="SE")
  # plt.plot(se_df['x+'], se_df['y+'], color ="yellow", label="pos+$\sigma$",alpha=0.3, linestyle='dashed')
  # plt.plot(se_df['x-'], se_df['y-'], color ="yellow", label="pos-$\sigma$", alpha=0.3, linestyle='dashed')

plt.legend()
plt.show()